In [1]:
# Inicialización en Python
from pyspark.sql import SparkSession
import mlflow
import mlflow.spark # Importante para loggear modelos de Spark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Iniciar Spark (Driver Program)
spark = SparkSession.builder.appName("MLOpsSparkML")     .getOrCreate()

# 2. Configurar MLflow Tracking (Ejemplo local en un directorio)
mlflow.set_tracking_uri("http://localhost:5000")

# 3. Definir el experimento activo
mlflow.set_experiment("Clasificacion_Ventas_Inicial")

print("Spark y MLflow listos para escalar.")

/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 16:46:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark y MLflow listos para escalar.


In [2]:
# -----------------------------------------------------------------
# --- Ejemplo 1: Pipeline Completo (Estimators) + Log con MLflow ---
# -----------------------------------------------------------------
print("\n--- Iniciando Ejemplo 1: Pipeline Completo ---")

# 4. Crear datos de ejemplo
# Queremos predecir si un cliente comprará (label)
# basado en su departamento (categórico) y edad (numérico).
datos_entrenamiento = [
    (0, "Ventas", 30, 1),  # id, depto, edad, si_compra (label=1)
    (1, "IT", 45, 1),
    (2, "Ventas", 22, 0),  # no_compra (label=0)
    (3, "Mktg", 38, 0),
    (4, "IT", 51, 1),
    (5, "Mktg", 29, 1),
    (6, "Ventas", 40, 0),
    (7, "IT", 33, 1)
]

columnas = ["id", "departamento", "edad", "label"]
df_train = spark.createDataFrame(datos_entrenamiento, columnas)


--- Iniciando Ejemplo 1: Pipeline Completo ---


In [3]:
df_train.show()

+---+------------+----+-----+
| id|departamento|edad|label|
+---+------------+----+-----+
|  0|      Ventas|  30|    1|
|  1|          IT|  45|    1|
|  2|      Ventas|  22|    0|
|  3|        Mktg|  38|    0|
|  4|          IT|  51|    1|
|  5|        Mktg|  29|    1|
|  6|      Ventas|  40|    0|
|  7|          IT|  33|    1|
+---+------------+----+-----+



In [29]:

# 5. Definir la Pipeline (una secuencia de Estimators y Transformers)

# --- Etapas de Preprocesamiento (Estimators) ---
# Un Estimator debe "aprender" de los datos (.fit())

# Estimator 1: Aprende el mapeo de strings a números (ej. "Ventas"->0.0, "IT"->1.0)
indexer = StringIndexer(inputCol="departamento", outputCol="depto_idx",handleInvalid='keep')

# Estimator 2: Aprende a convertir esos números en vectores one-hot
encoder = OneHotEncoder(inputCol="depto_idx", outputCol="depto_vec")


# --- Etapa de Ensamblaje (Transformer) ---
# Un Transformer solo aplica una regla fija (.transform())

# Transformer: Ensambla las columnas de features en un solo vector.
# No "aprende" nada, solo sigue la instrucción de juntar.
assembler = VectorAssembler(
    inputCols=["depto_vec", "edad"],
    outputCol="features"
)

assembler

VectorAssembler_22c247c7f1e0

In [30]:




# --- Etapa del Modelo (Estimator) ---
# El Estimator principal: El algoritmo de Machine Learning
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# 6. Encapsular todo en una Pipeline
# Una Pipeline es un "Estimator" grande que contiene otras etapas.
pipeline = Pipeline(stages=[indexer, encoder, assembler, lr])

# 7. 🚀 Entrenar y Registrar el Modelo con MLflow
with mlflow.start_run() as run:
    print(f"Iniciando Run de MLflow: {run.info.run_id}")

    # --- El corazón del proceso: .fit() ---
    # Al llamar a .fit() en la Pipeline (un Estimator):
    # 1. Se llama a .fit() en 'indexer' -> Se crea un 'StringIndexerModel' (Transformer)
    # 2. Se llama a .fit() en 'encoder' -> Se crea un 'OneHotEncoderModel' (Transformer)
    # 3. Se llama a .transform() en 'assembler' (ya es Transformer)
    # 4. Se llama a .fit() en 'lr' -> Se crea un 'LogisticRegressionModel' (Transformer)
    
    # El resultado es un "PipelineModel", que es un TRANSFORMER.
    pipeline_model = pipeline.fit(df_train)
    print("Modelo entrenado (PipelineModel creado).")

    # 8. 📦 Registrar el modelo (el Transformer) en MLflow
    # MLflow sabe cómo guardar, serializar y versionar modelos de Spark.
    mlflow.spark.log_model(
        spark_model=pipeline_model,
        artifact_path="modelo-clasificacion-ventas"
    )

    # Registrar parámetros
    mlflow.log_param("max_iter_lr", 10)
    mlflow.log_param("total_etapas", len(pipeline.getStages()))
    
    # Evaluar el modelo y registrar métricas
    predicciones_train = pipeline_model.transform(df_train)
    evaluador = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")


    auc = evaluador.evaluate(predicciones_train)
    evaluador_pr=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderPR")
    auc_pr = evaluador_pr.evaluate(predicciones_train)
    mlflow.log_metric("auc_entrenamiento", auc)
    mlflow.log_metric("auc_pr",auc_pr)
    
    print(f"Métrica (AUC): {auc}")
    print(f"Modelo y métricas registradas en MLflow (Run ID: {run.info.run_id})")

# 9. 🔮 Usar el Modelo (Transformer) cargado desde MLflow para Inferencia
print("\n--- Cargando modelo desde MLflow para predicción ---")

# Simular datos nuevos (sin la columna "label")
datos_nuevos = [
    (10, "IT", 33),     # Un IT de 33 años
    (11, "Datos", 48), # Un Ventas de 48
    (12, "Mktg", 21)    # Un Mktg de 21
]
df_nuevos = spark.createDataFrame(datos_nuevos, ["id", "departamento", "edad"])

# Cargar el PipelineModel (Transformer) guardado
modelo_cargado = mlflow.spark.load_model(
    f"runs:/{run.info.run_id}/modelo-clasificacion-ventas"
)

# Aplicar la transformación (predicción)
# Llama a .transform() en todas las etapas del PipelineModel
predicciones_nuevas = modelo_cargado.transform(df_nuevos)

predicciones_nuevas.select("id", "departamento", "edad", "prediction").show()

Iniciando Run de MLflow: 4be6e63afa484e689f3aa3ecc8379efb
Modelo entrenado (PipelineModel creado).
Métrica (AUC): 0.8666666666666667
Modelo y métricas registradas en MLflow (Run ID: 4be6e63afa484e689f3aa3ecc8379efb)
🏃 View run peaceful-bug-423 at: http://localhost:5000/#/experiments/1/runs/4be6e63afa484e689f3aa3ecc8379efb
🧪 View experiment at: http://localhost:5000/#/experiments/1

--- Cargando modelo desde MLflow para predicción ---


2025/11/10 17:20:47 INFO mlflow.spark: URI 'runs:/4be6e63afa484e689f3aa3ecc8379efb/modelo-clasificacion-ventas/sparkml' does not point to the current DFS.
2025/11/10 17:20:47 INFO mlflow.spark: File 'runs:/4be6e63afa484e689f3aa3ecc8379efb/modelo-clasificacion-ventas/sparkml' not found on DFS. Will attempt to upload the file.


+---+------------+----+----------+
| id|departamento|edad|prediction|
+---+------------+----+----------+
| 10|          IT|  33|       1.0|
| 11|       Datos|  48|       1.0|
| 12|        Mktg|  21|       1.0|
+---+------------+----+----------+



In [24]:
predicciones_train.select("label").toPandas()

,label
0,1
1,1
2,0
3,0
4,1
5,1
6,0
7,1


In [25]:
pipeline_model.stages

[StringIndexerModel: uid=StringIndexer_93fad8badc3d, handleInvalid=keep,
 OneHotEncoderModel: uid=OneHotEncoder_3f8af90075e4, dropLast=true, handleInvalid=error,
 VectorAssembler_91ae1be3bee4,
 LogisticRegressionModel: uid=LogisticRegression_dc0089b308da, numClasses=2, numFeatures=4]